# QuickCheck

[QuickCheck] is a property-based testing framework for Haskell.

[QuickCheck]: https://hackage.haskell.org/package/QuickCheck

> Authored by [Yoo Chung](mailto:chungyc@google.com)<br/>
> Written on 2023-11-30
>
> Copyright 2023 Google LLC.<br/>
> SPDX-License-Identifier: Apache-2.0

## Property-based testing

Instead of testing individual test cases, specify properties and check whether random inputs satisfy the properties.

## The Example

Every introduction to QuickCheck seems to start out with an example with a reverse function.
Let's define an `intReverse` function, just so we don't have to qualify with a concrete type everywhere.

In [ ]:
intReverse :: [Int] -> [Int]
intReverse = reverse

Obviously, we need to import the QuickCheck module.

In [ ]:
import Test.QuickCheck

We can then define a property for `intReverse` and have QuickCheck test it.
Every introduction to QuickCheck also seems to give as an example that reversing a list twice will give back the original list.

In [ ]:
prop_reverse xs = intReverse (intReverse xs) == xs

quickCheck prop_reverse

QuickCheck generates random values and tests that the property holds for these values.  You can see what values it tests the property with using [`verbose`](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#v:verbose).

In [ ]:
quickCheck $ verbose prop_reverse

## A strategy for property-based testing

When I first encountered The Example in introductions to QuickCheck, I was awestruck at how it could test general properties instead of just tests with specific values.

After some time, I was less impressed with The Example, because while it demonstrates how QuickCheck can check general properties, it's rather lacking in regards to testing whether a function behaves _correctly_.  It will happily pass the test for a not quite correct implementation of a reverse function.  In fact, it will pass for the `id` function.

In [ ]:
almostReverse xs | sum xs > 15 && sum xs < 19 = xs
                 | otherwise = intReverse xs

quickCheck (\xs -> almostReverse (almostReverse xs) === xs)

quickCheck (\xs -> id (id xs) === xs)

An early strategy I sometimes used was to compare the function being tested against a naive and "obviously" correct implementation.  For example:

In [ ]:
naiveReverse :: [Int] -> [Int]
naiveReverse [] = []
naiveReverse (x:xs) = naiveReverse xs ++ [x]

quickCheck (\xs -> intReverse xs === naiveReverse xs)

This never sat with me very well.  At some point, I began testing using induction.  In other words, I would test properties for the base case, and then check the property holds for a non-base case assuming it holds for a smaller case.  For example:

In [ ]:
prop_reverse' = intReverse [] === []
prop_reverse'' x xs = intReverse (x:xs) === intReverse xs ++ [x]

quickCheck prop_reverse'
quickCheck prop_reverse''

This style of property-based testing will make it much more likely to catch bugs than The Example.  For example:

In [ ]:
prop_badReverse = almostReverse [] === []
prop_badReverse' x xs = almostReverse (x:xs) === almostReverse xs ++ [x]

quickCheck prop_badReverse
quickCheck prop_badReverse'

### Exercise

Let's say you have a brand new and shiny implementation of a list concatenation function.
What properties would you test for make sure it is correct?

In [ ]:
intConcat :: [Int] -> [Int] -> [Int]
intConcat = (++)

-- Implement your properties here and test "intConcat" using "quickCheck".


## Testing function inputs

Being a functional programming language, we should be able to test properties of functions which have other functions as arguments.  Unsurprisingly, QuickCheck has the ability to generate random functions as input to a function being tested.

The [`Fun`](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#t:Fun) type with the constructor `Fun` can be used to generate functions (to be clear, one could generate functions in other ways, too).  The `Fun` constructor has auxillary data and a function itself, so one can pattern match to get the randomly generated function.  Alternatively, [`applyFun`](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#v:applyFun) can be used to extract the function from a value of type `Fun`.

In [ ]:
prop_map :: Fun Integer String -> Property
prop_map fun = map (applyFun fun) [] === []

prop_map' :: Fun Integer String -> Integer -> [Integer] -> Property
prop_map' (Fun _ f) x xs = map f (x : xs) === (f x : map f xs)

quickCheck prop_map
quickCheck prop_map'

The function generated is in a showable form, which is why the function is not generated directly but instead generated as part of a `Fun` value, which also has the information to show the function.

In [ ]:
prop_notMap :: Fun Integer String -> [Integer] -> Property
prop_notMap (Fun _ f) xs = map f xs === ["1","2","3"]

quickCheck prop_notMap

### Exercise

Test that the function "filter" only returns elements in a list which return true when fed into a given function.

In [ ]:
-- Implement your properties here and test "filter" using "quickCheck".


## Counterexamples

Instead of generating random input directly, you may be generating multiple values which you combine into the actual input.  In this case, it may be harder to see for what value a property is failing, because by default QuickCheck only shows the values it generates directly.  [`counterexample`](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#v:counterexample) can be used to show the actual value being tested.  It could also be used to make it easier to see which values are what when there are many generated values for a property.

In [ ]:
prop_badReverse'' x xs =
    counterexample ("list is " ++ show (x:xs)) $
    almostReverse (x:xs) === almostReverse xs ++ [x]

quickCheck prop_badReverse''

## Generating inputs

How does QuickCheck actually generate its random inputs?

### Generators

QuickCheck uses the [`Gen`](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#t:Gen) monad to generate random inputs.  In the following, `choose (1,50)` is a value of `Gen`.

In [ ]:
quickCheck $ forAll (choose (1,50)) $ \x -> x > 0 && x < 51

You could also define a `Gen` value with its own name so that you could use it over and over again.

In [ ]:
isPrime :: Int -> Bool
isPrime n = not $ any (\m -> n `mod` m == 0) [2..n-1]

gen :: Gen Int
gen = do
  size <- getSize
  x <- choose (2, size + 2)
  y <- choose (2, size + 2)
  return $ x*y

quickCheck $ forAll gen $ \x -> not (isPrime x)

### Type-level modifiers

[Type-level modifiers](https://hackage.haskell.org/package/QuickCheck-2.14.3/docs/Test-QuickCheck.html#g:16) are a convenient way to generate values with certain properties.

In [ ]:
quickCheck $ \(Positive x) -> x - 10 < length [1,2,3]

In [ ]:
import Data.List (sort)

quickCheck $ \(Ordered xs) -> sort xs === xs

In [ ]:
quickCheck $ \(NonEmpty xs) -> total (head xs)

### `Arbitrary` typeclass

We did not specify how lists or numbers are to be generated in many of the examples.  How does QuickCheck know how to do this?  It's because QuickCheck makes many basic types into instances of the `Arbitrary` typeclass.  The typeclass specifies both how to generate values for a type, and also how to [shrink values](#Shrinking) so that a failing test case can be shown with a simpler value than the one where the failure was initially detected.

If you generate the same sort of values over and over again, it can be convenient to define your own type which is an instance of `Arbitrary`.  This is actually what type-level modifiers are.

In [ ]:
newtype Composite = Composite Int deriving (Eq, Ord, Show)

instance Arbitrary Composite where
  arbitrary = do
    size <- getSize
    x <- choose (2, size + 2)
    y <- choose (2, size + 2)
    return $ Composite $ x*y

  shrink (Composite n)
    | n <= 2 = []
    | [] <- smaller = []
    | otherwise = [Composite $ last smaller]
    where smaller = filter isPrime [2..n-1]
    
quickCheck $ \(Composite n) -> not (isPrime n)

quickCheck $ \(Composite m) -> \(Composite n) -> gcd m n /= 1

## Shrinking

QuickCheck can shrink random inputs on which a property fails, which can make it easier to debug with simpler input.

In [ ]:
quickCheck prop_badReverse''

In [ ]:
quickCheck $ noShrinking prop_badReverse''

## Haddock

[Haddock](https://haskell-haddock.readthedocs.io/) can document properties with the [`prop>`](https://haskell-haddock.readthedocs.io/en/latest/markup.html#properties) markup.  [doctest](https://hackage.haskell.org/package/doctest) can extract these properties to test that the properties are satisfied using QuickCheck.

In [ ]:
-- | Reverses a list of strings.
--
-- prop> (strReverse . strReverse) x == x
--
strReverse :: [String] -> [String]
strReverse = reverse

## Fuzzing

Generate random inputs for a function and check whether it behaves correctly ...  Isn't there another concept which sounds similar?

That's right, it's fuzzing!  For the most basic sort of fuzzing, i.e., checking whether a program crashes on random inputs, this would simply be testing the property that a function is total.

In [ ]:
quickCheck $ \xs -> total (head xs)

At least one security health scanning system considers property-based testing in Haskell to be fuzzing.  For example, see [OpenSSF Scorecard](https://securityscorecards.dev/).

## Other property-testing libraries

Focus on easier input generation:

* [Hedgehog](https://hackage.haskell.org/package/hedgehog)
* [Validity](https://hackage.haskell.org/package/validity)

Focus on exhaustive testing of small inputs instead of random inputs:

* [SmallCheck](https://hackage.haskell.org/package/smallcheck)

## Test frameworks

More like meta-frameworks, these make it easier to have tests from multiple frameworks such as [HUnit](https://hackage.haskell.org/package/HUnit) or QuickCheck.

*   [Hspec](https://hspec.github.io/)
*   [Tasty](https://hackage.haskell.org/package/tasty)

## License

Copyright 2023 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.